In [44]:
%load_ext autoreload
%autoreload 2
from corr_study.datasetApi import *
import h5py
import open3d as o3d
from tqdm import trange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
hf.close()

In [76]:
w = Weather.Clear
t = Time.Sunset

names = ["t3low", "t3medium", "t3high", "Town03LowNEW", "Town03MediumNEW", "Town03HighNEW"]
sensors = [Sensor.LT, Sensor.LFL, Sensor.LFR]

dataset = Dataset("corr_study/dataset/")

for n in names:
    print(n)
    for s in sensors:
        print(s)
        path_to_bbox = dataset.get_path_bbox(n, w, t)
        with h5py.File(path_to_bbox,'r') as f:
            root_grp = f.get("BBOX")
            ids = list(root_grp.keys())
            ids.sort()
            for id_actor in ids:
                agent = root_grp.get(id_actor)
                if "vehicle" in agent.attrs['type']:
                    ego = root_grp.get(id_actor)
                    ego_id = int(id_actor)
                    break
            loc = np.array(ego.get('location'))
            rot = np.array(ego.get('rotation'))
        ego_positions = {"loc":loc, "rot":rot}


        folder = dataset._get_path_h5_folder(n, w, t, s)
        hf = h5py.File(folder + "/fully_oriented.hdf5", "w")
        main_group = hf.create_group("data")
        len_dataset = dataset.get_measurement_series_length_TLC(n, w, t, s)
        for i in trange(len_dataset):
            spc = dataset.open_measurement_sample_TLC(n, w, t, s, i+1)
            pc = o3d.geometry.PointCloud()
            pc.points = o3d.utility.Vector3dVector(spc.data)
                

            T_sens = s.get_homogeneous_matrix()
            T_rot = np.eye(4)
            T_rot[:3, :3] = pc.get_rotation_matrix_from_xyz((np.radians(-ego_positions["rot"][i,2]),
                                                            np.radians(-ego_positions["rot"][i,0]), 
                                                            np.radians(-ego_positions["rot"][i,1])))
            T_loc = np.eye(4)
            T_loc[0:3,3] = [-ego_positions["loc"][i,1],
                            -ego_positions["loc"][i,0],
                            ego_positions["loc"][i,2]]
            T = np.dot(T_loc, T_rot)
            T = np.dot(T, T_sens)
            pc.transform(T)
            time_group_key = str(i+1).zfill(5)
            data = np.concatenate((np.asarray(pc.points), spc.ground_truth[:, [1, 0]]), axis=1)
            main_group.create_dataset(time_group_key, data=data)
        hf.close()


t3low
Sensor.LT


100%|██████████| 3254/3254 [03:32<00:00, 15.31it/s]


Sensor.LFL


100%|██████████| 3254/3254 [03:48<00:00, 14.27it/s]


Sensor.LFR


...

100%|██████████| 5400/5400 [08:07<00:00, 11.08it/s]


Sensor.LFL


100%|██████████| 5400/5400 [07:17<00:00, 12.35it/s]


Sensor.LFR


100%|██████████| 5400/5400 [08:10<00:00, 11.00it/s]


In [84]:
import seaborn as sns
w = Weather.Clear
t = Time.Sunset
s = Sensor.LT
name = "Town03HighNEW"
base = dataset._get_path_h5_folder(name, w, t, s) + "/preprocessed/"
llen = dataset.get_measurement_series_length_TLC(name, w, t, s)
pcds = []
print(llen)
p = sns.color_palette("rocket", n_colors=len(range(1, llen, 50)))
j=0
for i in range(1, llen, 50):
    pcd = o3d.io.read_point_cloud(base + str(i+1).zfill(5) + ".pcd")
    pcd.paint_uniform_color(p[j])
    j = j+1
    pcds.append(pcd)
# o3d.visualization.draw_geometries(pcds)


5400


In [78]:
import os
w = Weather.Clear
t = Time.Sunset

for name in ["t3low", "t3medium", "t3high", "Town03LowNEW", "Town03MediumNEW", "Town03HighNEW"]:
    for s in [Sensor.LT, Sensor.LFL, Sensor.LFR]:
        path_to_bbox = dataset.get_path_bbox(name, w, t)
        with h5py.File(path_to_bbox,'r') as f:
            root_grp = f.get("BBOX")
            ids = list(root_grp.keys())
            ids.sort()
            for id_actor in ids:
                agent = root_grp.get(id_actor)
                if "vehicle" in agent.attrs['type']:
                    ego = root_grp.get(id_actor)
                    ego_id = int(id_actor)
                    break

        len_dataset = dataset.get_measurement_series_length_TLC(name, w, t, s)
        base = dataset._get_path_h5_folder(name, w, t, s) + "/preprocessed/"
        os.mkdir(base)
        for i in trange(len_dataset):
            pcs = dataset.open_measurement_sample_TLC_fully_oriented(name, w, t, s, i+1)
            pcs.data = pcs.data[pcs.ground_truth[:,1] != ego_id]
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(pcs.data)
            o3d.io.write_point_cloud(base + str(i+1).zfill(5) + ".pcd", pcd)

100%|██████████| 5400/5400 [01:50<00:00, 48.82it/s]
